In [2]:
import numpy as np
import random
from qutip import *
import matplotlib.pyplot as plt

# Homework 5

### 1a)

p2 can only be as large as the channel efficiency number to guarantee secrecy from a spy. Any larger value would allow Eve to conduct a photon splitting attack while still transmitting enough photons within reason. When p2 < channel efficiency, it becomes very clear if there is a spy because a number of photons is significantly lower than what would be expected to a point of statistical improbability. Therefore in this channel, p2 must be less than .1 to ensure the detection of Eve.

### 1b)

Alice Definition:

In [125]:
def Alice(Np, p2):
    #output a list of Np rows, 2 columns
    #first col = index, second col = qubit in polarization of sent photon (in H/V basis) - random
    result = []
    for a in range(Np):
        count = 1
        val = random.random() #randomly assigns horizontal or diagonal basis
        if val >= .5: #if alice chooses horizontal basis
            temp_ket = basis(2,0) #|0>
        else: #if alice chooses diagonal basis
            temp_ket = (basis(2,0) + basis(2,1)).unit() #|+>
        if random.random() < p2:
            count = 2
        result.append([a,temp_ket, count])
    return result

alice_res = Alice(8, .1)

# Print alice data (This printing/formatting code below is made with ChatGPT's help)
print("Alice's Photons Sent:")
print("{:<10} {:<15} {:<20} {:<25}".format('Photon', 'Bit Sent', '# of Photons sent', 'Quantum State'))
print('-' * 50)  # Separator line
for idx, state, count in alice_res:
    # Determine the bit Alice sent based on the state
    if state == basis(2, 0):
        bit_sent = 0
    else:
        bit_sent = 1
    # Convert the quantum state to a string representation
    state_str = state.full().flatten()
    state_str_formatted = f"[{state_str[0]:.2f}, {state_str[1]:.2f}]"
    # Print the index, bit sent, and the quantum state
    print("{:<10} {:<15} {:<20} {:<25}".format(idx, bit_sent, count, state_str_formatted))

Alice's Photons Sent:
Photon     Bit Sent        # of Photons sent    Quantum State            
--------------------------------------------------
0          1               1                    [0.71+0.00j, 0.71+0.00j] 
1          1               1                    [0.71+0.00j, 0.71+0.00j] 
2          1               1                    [0.71+0.00j, 0.71+0.00j] 
3          1               1                    [0.71+0.00j, 0.71+0.00j] 
4          1               2                    [0.71+0.00j, 0.71+0.00j] 
5          0               2                    [1.00+0.00j, 0.00+0.00j] 
6          1               1                    [0.71+0.00j, 0.71+0.00j] 
7          0               1                    [1.00+0.00j, 0.00+0.00j] 


Eve definition:

In [126]:
def Eve(alice_arr, strat_check):
    ket_one = basis(2,1) #|1>
    ket_zero = basis(2,0) #|0>
    ket_minus = (basis(2,0) - basis(2,1)).unit() #|->
    ket_plus = (basis(2,0) + basis(2,1)).unit() #|+>
    result = []
    for photon in alice_arr:
        ind, ket, photon_count = photon
        if photon_count == 2: #if 2 photons sent, we can perform photon splitting attack, otherwise we will do nothing
            result.append([ind, ket,1])
        if photon_count ==1 and strat_check==True:
            val = random.random()
            if val >= .5: #if Eve chooses vertical basis
                #probabilty we get a click = |<1|psi>|^2
                prob = np.abs(ket_one.dag() * ket) ** 2
                if prob >= .499: #if the click prob >= 50%, we know that alice chose diagonal basis and that she sent a 1
                    result.append([ind, ket_plus,1]) #we recreate the plus state we know alice sent
                else:
                    rand = random.random() #if eve does not know what is sent, randomly choose between |0> and |+> to send to bob
                    if rand >= 0.5:
                        choice = ket_zero
                    else:
                        choice = ket_plus
                    result.append([ind, choice,1])  
            else: #if Eve chooses diagonal basis
                prob = np.abs(ket_minus.dag() * ket) ** 2
                if prob >= .499: #if the click prob >= 50%, we know that alice chose horizontal basis and that she sent a 0
                    result.append([ind, ket_zero,1])
                else:
                    rand = random.random() #if eve does not know what is sent, randomly choose between |0> and |+> to send to bob
                    if rand >= 0.5:
                        choice = ket_zero
                    else:
                        choice = ket_plus
                    result.append([ind, choice,1])
    return result  


Bob Definition:

In [127]:
def Bob(alice_arr, efficiency):
    ket_one = basis(2,1) #|1>
    ket_minus = (basis(2,0) - basis(2,1)).unit() #|->
    result = []
    for photon in alice_arr:
        ind, ket, count = photon
        if random.random() > efficiency: #allows for us to test various channel efficiency vals if wanted
            continue
        val2 = random.random()
        if val2 >= .5: #if Bob chooses vertical basis
            #probabilty we get a click = |<1|ket>|^2
            prob = np.abs(ket_one.dag() * ket) ** 2
            if prob >= .499: #if the click prob >= 50%, we know that alice chose diagonal basis and that she sent a 1
                result.append([ind, 1])

        else: # if Bob chooses diagonal basis
            prob = np.abs(ket_minus.dag() * ket) ** 2
            if prob >= .499: #if the click prob >= 50%, we know that alice chose horizontal basis and that she sent a 0
                result.append([ind, 0])
    return result

Analyze p2=.2 results

In [128]:
N = 1000
p2 = .2
#If eve present case
alice_100 = Alice(N,.2)
eve_output_split_only = Eve(alice_100, False)
bob_output_eve_split = Bob(eve_output_split_only, .1)
eve_output = Eve(alice_100, True)
bob_output_eve = Bob(eve_output, .1)
#if eve not present case
bob_output = Bob(alice_100, .1)
#print("Bob's output with no Eve", bob_output)
#print("Bob's output with Eve interference", bob_output_eve)
#error rate calculations
alice_100_readable = []
for ind, state, count in alice_100:
    if state == basis(2,0): 
        alice_100_readable.append([ind, 0]) #if we are in Horizontal state she sends 0
    else:
        alice_100_readable.append([ind, 1]) #if we are in + state she sends 1

accepted_eve = []
for ind, bob_meas in bob_output_eve:
    accepted_eve.append([ind, alice_100_readable[ind][1], bob_meas]) #array holds index, alice measurement, bob measurement
accepted = []
for ind, bm in bob_output:
    accepted.append([ind, alice_100_readable[ind][1], bm])
accepted_eve_split = []
for ind, bob_meas in bob_output_eve_split:
    accepted_eve_split.append([ind, alice_100_readable[ind][1], bob_meas]) #array holds index, alice measurement, bob measurement
errors_eve = sum(1 for ind, a_bit, b_bit in accepted_eve if a_bit!=b_bit) #if the bit alice outputted != the bit bob recieved we know theres an error caused by eve (that rhymes!)
error_rate_eve = round(errors_eve/len(accepted_eve),3)
errors = sum(1 for ind, a_bit, b_bit in accepted if a_bit!=b_bit) #if the bit alice outputted != the bit bob recieved we know theres an error caused by eve (that rhymes!)
error_rate = round(errors/len(accepted),3)
errors_eve_split = sum(1 for ind, a_bit, b_bit in accepted_eve_split if a_bit!=b_bit) #if the bit alice outputted != the bit bob recieved we know theres an error caused by eve (that rhymes!)
error_rate_eve_split = round(errors_eve_split/len(accepted_eve_split),3)

print("Error rate with Eve's presence: ", error_rate_eve, "; Amount of measurements made by Bob: ", len(bob_output_eve)) #if you want to see Eve's hybrid strategy
print("Error rate without Eve", error_rate, "; Amount of measurements made by Bob: ", len(bob_output))
print("Error rate with Eve's presence (only splitting): ", error_rate_eve_split, "; Amount of measurements made by Bob: ", len(bob_output_eve_split))

        

Error rate with Eve's presence:  0.151 ; Amount of measurements made by Bob:  53
Error rate without Eve 0.0 ; Amount of measurements made by Bob:  55
Error rate with Eve's presence (only splitting):  0.0 ; Amount of measurements made by Bob:  14


Here, while it is clear that the # of photons Bob recieves if eve only sends photons from photon splitting attacks is less than expected, it is still generally within reason. It is feasibly possible for Eve to conduct a ohoton splitting attack if p2 = .2

Analyzing p2=.05 results

In [129]:
N = 1000
p2 = .05
#If eve present case
alice_100 = Alice(N,p2)
eve_output_split_only = Eve(alice_100, False)
bob_output_eve_split = Bob(eve_output_split_only, .1)
eve_output = Eve(alice_100, True)
bob_output_eve = Bob(eve_output, .1)
#if eve not present case
bob_output = Bob(alice_100, .1)
#print("Bob's output with no Eve", bob_output)
#print("Bob's output with Eve interference", bob_output_eve)
#error rate calculations
alice_100_readable = []
for ind, state, count in alice_100:
    if state == basis(2,0): 
        alice_100_readable.append([ind, 0]) #if we are in Horizontal state she sends 0
    else:
        alice_100_readable.append([ind, 1]) #if we are in + state she sends 1

accepted_eve = []
for ind, bob_meas in bob_output_eve:
    accepted_eve.append([ind, alice_100_readable[ind][1], bob_meas]) #array holds index, alice measurement, bob measurement
accepted = []
for ind, bm in bob_output:
    accepted.append([ind, alice_100_readable[ind][1], bm])
accepted_eve_split = []
for ind, bob_meas in bob_output_eve_split:
    accepted_eve_split.append([ind, alice_100_readable[ind][1], bob_meas]) #array holds index, alice measurement, bob measurement
errors_eve = sum(1 for ind, a_bit, b_bit in accepted_eve if a_bit!=b_bit) #if the bit alice outputted != the bit bob recieved we know theres an error caused by eve (that rhymes!)
error_rate_eve = round(errors_eve/len(accepted_eve),3)
errors = sum(1 for ind, a_bit, b_bit in accepted if a_bit!=b_bit) #if the bit alice outputted != the bit bob recieved we know theres an error caused by eve (that rhymes!)
error_rate = round(errors/len(accepted),3)
errors_eve_split = sum(1 for ind, a_bit, b_bit in accepted_eve_split if a_bit!=b_bit) #if the bit alice outputted != the bit bob recieved we know theres an error caused by eve (that rhymes!)
error_rate_eve_split = round(errors_eve_split/len(accepted_eve_split),3)

#print("Error rate with Eve's presence: ", error_rate_eve, "; Amount of measurements made by Bob: ", len(bob_output_eve))
print("Error rate without Eve", error_rate, "; Amount of measurements made by Bob: ", len(bob_output))
print("Error rate with Eve's presence (only splitting): ", error_rate_eve_split, "; Amount of measurements made by Bob: ", len(bob_output_eve_split))

        

Error rate without Eve 0.0 ; Amount of measurements made by Bob:  52
Error rate with Eve's presence (only splitting):  0.0 ; Amount of measurements made by Bob:  1


Here, it is very clear when Eve has executed a photon splitting attack. She will be caught if trying to do so since Bob will accept barely any photons when she is doing so. Eve cannot conduct a photon splitting attack when p2=.05

### Problem 2) Decoy State Protocol

In [166]:
def Alice(Np, p2, decoy_percent):
    #output a list of Np rows, 2 columns
    #first col = index, second col = qubit in polarization of sent photon (in H/V basis) - random
    result = []
    for a in range(Np):
        count = 1
        decoy_state = (random.random() < decoy_percent) #given some % of states that are decoys
        val = random.random() #randomly assigns horizontal or diagonal basis
        if val >= .5: #if alice chooses horizontal basis
            temp_ket = basis(2,0) #|0>
        else: #if alice chooses diagonal basis
            temp_ket = (basis(2,0) + basis(2,1)).unit() #|+>
        if decoy_state ==True or random.random() < p2 : #all decoy states are multi-photon
            count = 2
        result.append([a,temp_ket, count, decoy_state])
    return result

def Eve(alice_arr):
    ket_one = basis(2,1) #|1>
    ket_zero = basis(2,0) #|0>
    ket_minus = (basis(2,0) - basis(2,1)).unit() #|->
    ket_plus = (basis(2,0) + basis(2,1)).unit() #|+>
    result = []
    for photon in alice_arr:
        ind, ket, photon_count, filler = photon #we'll name the decoy filler here since Eve will not know about the decoys. 
        if photon_count == 2: #if 2 photons sent, we can perform photon splitting attack, otherwise we will do nothing
            result.append([ind, ket,1,filler])
        # else:
        #     val = random.random()
        #     if val >= .5: #if Eve chooses vertical basis
        #         #probabilty we get a click = |<1|psi>|^2
        #         prob = np.abs(ket_one.dag() * ket) ** 2
        #         if prob >= .499: #if the click prob >= 50%, we know that alice chose diagonal basis and that she sent a 1
        #             result.append([ind, ket_plus,1,filler]) #we recreate the plus state we know alice sent
        #         else:
        #             rand = random.random() #if eve does not know what is sent, randomly choose between |0> and |+> to send to bob
        #             if rand >= 0.5:
        #                 choice = ket_zero
        #             else:
        #                 choice = ket_plus
        #             result.append([ind, choice,1,filler])  
        #     else: #if Eve chooses diagonal basis
        #         prob = np.abs(ket_minus.dag() * ket) ** 2
        #         if prob >= .499: #if the click prob >= 50%, we know that alice chose horizontal basis and that she sent a 0
        #             result.append([ind, ket_zero,1,filler])
        #         else:
        #             rand = random.random() #if eve does not know what is sent, randomly choose between |0> and |+> to send to bob
        #             if rand >= 0.5:
        #                 choice = ket_zero
        #             else:
        #                 choice = ket_plus
        #             result.append([ind, choice,1,filler])
    return result  

 
def Bob(alice_arr, efficiency):
    ket_one = basis(2,1) #|1>
    ket_minus = (basis(2,0) - basis(2,1)).unit() #|->
    result = []
    for photon in alice_arr:
        ind, ket, count, is_decoy = photon
        if random.random() > efficiency: #allows for us to test various channel efficiency vals if wanted
            continue
        val2 = random.random()
        if val2 >= .5: #if Bob chooses vertical basis
            #probabilty we get a click = |<1|ket>|^2
            prob = np.abs(ket_one.dag() * ket) ** 2
            if prob >= .499: #if the click prob >= 50%, we know that alice chose diagonal basis and that she sent a 1
                result.append([ind, 1, is_decoy])

        else: # if Bob chooses diagonal basis
            prob = np.abs(ket_minus.dag() * ket) ** 2
            if prob >= .499: #if the click prob >= 50%, we know that alice chose horizontal basis and that she sent a 0
                result.append([ind, 0, is_decoy])
    return result



In [175]:
N = 10000
p2 = .2 #we'll set p2=.2 for this example, normally Alice would not be able to detect Eve with this p2 value
decoy_percent = .1 #10% percent of states will be 2 photon decoy states
alice_output = Alice(N,p2,decoy_percent)
eve_output = Eve(alice_output)
bob_output_eve = Bob(eve_output, .1)
#now we want to measure the channel efficiencies for the decoy states and non-decoy states
general_channel_eff = ((len(bob_output_eve))*2)/N
print("Overall channel efficiency:", general_channel_eff)

amt_decoy_sent = 0
for ind, state, count, decoy in alice_output:
    if decoy == True:
        amt_decoy_sent +=1
amt_decoy_recieved = 0
for ind, state, decoy in bob_output_eve:
    if decoy == True:
        amt_decoy_recieved +=1
decoy_channel_eff = (amt_decoy_recieved*2)/amt_decoy_sent
print("Decoy channel efficiency:", decoy_channel_eff)

Overall channel efficiency: 0.0312
Decoy channel efficiency: 0.10854271356783919


Here, it is clear that if Eve is implementing a photon splitting attack because the decoy channel efficiency is consistely 3-4 times larger than the overall channel efficiency. This allows us to tell when a photon splitting attack is occuring even if p2>channel efficiency.